## Interval Analysis

In [26]:
# !pip install tensorboardX

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import time
import matplotlib.pyplot as plt
from bound_propagation import BoundModelFactory, HyperRectangle

from torchvision import datasets, transforms

# from tensorboardX import SummaryWriter

use_cuda = False
device = torch.device("cuda" if use_cuda else "cpu")
batch_size = 64

np.random.seed(42)
torch.manual_seed(42)


## Dataloaders
train_dataset = datasets.MNIST(
    "mnist_data/",
    train=True,
    download=True,
    transform=transforms.Compose([transforms.ToTensor()]),
)
test_dataset = datasets.MNIST(
    "mnist_data/",
    train=False,
    download=True,
    transform=transforms.Compose([transforms.ToTensor()]),
)

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True
)
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False
)


## Simple NN. You can change this if you want. If you change it, mention the architectural details in your report.
class Normalize(nn.Sequential):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return (x - 0.1307) / 0.3081


class Net(nn.Sequential):
    def __init__(self):
        super(Net, self).__init__(
            Normalize(), nn.Linear(28 * 28, 200), nn.ReLU(), nn.Linear(200, 10)
        )

    def forward(self, x):
        x = x.view((-1, 28 * 28))
        x = super().forward(x)
        x = F.softmax(x, dim=-1)  # added softmax for probabilities
        return x


# Add the data normalization as a first "layer" to the network
# this allows us to search for adverserial examples to the real image, rather than
# to the normalized image
model = nn.Sequential(Normalize(), Net())

model = model.to(device)
model.train()

Sequential(
  (0): Normalize()
  (1): Net(
    (0): Normalize()
    (1): Linear(in_features=784, out_features=200, bias=True)
    (2): ReLU()
    (3): Linear(in_features=200, out_features=10, bias=True)
  )
)

In [27]:
def train_model(model, num_epochs):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            images, labels = data
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.3f}')

def test_model(model):
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for data in test_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        print(f'Accuracy on images: {100 * correct / total}')

In [28]:
train_model(model, 10)
test_model(model)

Epoch 1/10, Loss: 1.718
Epoch 2/10, Loss: 1.570
Epoch 3/10, Loss: 1.552
Epoch 4/10, Loss: 1.541
Epoch 5/10, Loss: 1.534
Epoch 6/10, Loss: 1.528
Epoch 7/10, Loss: 1.523
Epoch 8/10, Loss: 1.519
Epoch 9/10, Loss: 1.516
Epoch 10/10, Loss: 1.513
Accuracy on images: 95.11


### Write the interval analysis for the simple model

In [52]:
## TODO: Write the interval analysis for the simple model
## you can use https://github.com/Zinoex/bound_propagation
#  !pip install bound-propagation
from bound_propagation import BoundModelFactory, HyperRectangle


net = BoundModelFactory().build(model)


def interval_analysis(net, test_loader, epsilons):
    with torch.no_grad():
        for e in epsilons:
            correct = 0
            total = 0
            for imgs, labels in test_loader:
                box = HyperRectangle.from_eps(imgs.view(imgs.size(0), -1), e)
                out = net.crown_ibp(box, alpha=True).concretize()
                lo, hi = out.lower, out.upper

                labs = labels.cpu().numpy()
                bad_mask = torch.ones_like(hi, dtype=torch.bool, device=hi.device)
                for idx, lab in enumerate(labs):
                    bad_mask[idx, lab] = False
                lo_true = lo[range(len(labels)), labs]
                hi_bad = hi.masked_select(bad_mask).view(len(labels), -1)
                correct += (lo_true > hi_bad.max(dim=1).values).sum().item()
                total += len(labels)
            print(f"Epsilon: {e}, Robust Accuracy: {100 * correct / total:.2f}%")


epsilons = np.linspace(0.01, 0.1, 10)
interval_analysis(net, test_loader, epsilons)

Epsilon: 0.01, Robust Accuracy: 70.27%
Epsilon: 0.020000000000000004, Robust Accuracy: 15.34%
Epsilon: 0.030000000000000006, Robust Accuracy: 1.17%
Epsilon: 0.04000000000000001, Robust Accuracy: 0.02%
Epsilon: 0.05000000000000001, Robust Accuracy: 0.00%
Epsilon: 0.06000000000000001, Robust Accuracy: 0.00%
Epsilon: 0.07, Robust Accuracy: 0.00%
Epsilon: 0.08, Robust Accuracy: 0.00%
Epsilon: 0.09000000000000001, Robust Accuracy: 0.00%
Epsilon: 0.1, Robust Accuracy: 0.00%
